# Notebook - Dallas Morning News Case Study

The Dallas Morning News [uncovers a cheating scandal on the TAKS test](http://clipfile.org/?p=754). 

_note: Try not to consult the clip above unless you're stuck. The clip also doesn't contain all of the answers so there is still plenty to investigate._

You're a reporter at the Dallas Morning News in 2005. While talking to some teachers at your local school district, you recieved a tip that a fourth grade classroom at Harrell Budd Elementary in southern Dallas has unusually high test scores in reading on the TAKS test (the state's standardized test to measure student progress). To check out this claim you've downloaded data from the Texas Education Agency. To dig deeper into this, you have acquired records from the district of average standardized TAKS test scores for reading and math for third, fourth and fifth graders for the previous year. You also decide to download the reading scores for 2003.

Let's see how the Dallas Morning News used regressions to find evidence of this claim and try to find out if it is an isolated case.

### Prep work: Downloading necessary files
Before we get started, we need to download all of the data we'll be using.
* **cfy04e4.dat:** 2004 fourth-grade TAKS scores - standardized test scores for 2004's fourth-graders
* **cfy04e5.dat:** 2004 fifth-grade TAKS scores - standardized test scores for 2004's fifth-graders
* **cfy04e3.dat:** 2004 third-grade TAKS scores - standardized test scores for 2004's third-graders

More data you might want
* **cfy03e3.dat:** 2003 third-grade TAKS scores - standardized test scores for 2003's third-graders

Data Documentation: 
https://tea.texas.gov/student-assessment/testing/taks-resources/taks-variables-formats-and-descriptions


In [1]:
# # Make data directory if it doesn't exist
# # (you can remove the comments, run this cell once to download the data, and add the comments back)
# !mkdir -p data
# !wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/dmn-texas-school-cheating/data/cfy04e4.dat -P data
# !wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/dmn-texas-school-cheating/data/cfy03e3.dat -P data
# !wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/dmn-texas-school-cheating/data/cfy04e5.dat -P data
# !wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/dmn-texas-school-cheating/data/cfy04e3.dat -P data

### R and Python Imports

In [2]:
%reload_ext rpy2.ipython
%reload_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

In [5]:
%%R

require('tidyverse')

In [4]:
import pandas as pd
import numpy as np

# Show a large number of rows and columns when displaying dfs
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
from IPython.display import display, HTML

# Finding suspicious datapoints with linear regression

Can we see evidence of possible cheating at Harrell Budd Elementary? Is Harrell Budd Elementary's fourth grade reading classrooms the only place where we see signs of possible cheating in 2004? 

How else can you design regressions to help you uncover other instances of cheating? 

In the cells below, try to define your own questions and see if you can uncover other suspicious scores using regressions. You may also want to use plots as necessary. You can start with reading (`r_all_rs`) and math (`m_all_rs`) scores in the data we have loaded, but if you'd like, you're also welcome to bring in other variables from the [data documentation](
https://tea.texas.gov/sites/default/files/TAKS-Variables-prior2005.pdf).

- If you've got an interesting finding, put up a green sticky note! 🟩
- If you're stuck, put up a red sticky note! 🟥

## Load the data 
We'll start by opening up our dataset - standardized test performance at each school. We have:
- 3rd 4th and 5th grade reading and math scores for 2004 
- 4th grade reading and math scores for 2003

In [10]:
df_third_grade_2004 = pd.read_csv("data/cfy04e3.dat", usecols=['r_all_rs', 'm_all_rs', 'CNAME', 'CAMPUS']).set_index('CAMPUS').add_suffix('_third')
df_fourth_grade_2004 = pd.read_csv("data/cfy04e4.dat", usecols=['r_all_rs', 'm_all_rs', 'CNAME', 'CAMPUS']).set_index('CAMPUS').add_suffix('_fourth')
df_fifth_grade_2004 = pd.read_csv("data/cfy04e5.dat", usecols=['r_all_rs', 'm_all_rs', 'CNAME', 'CAMPUS']).set_index('CAMPUS').add_suffix('_fifth')

# This time I'm also adding fourth grade scores from last year (2003)! 
df_fourth_grade_2003 = pd.read_csv("data/cfy04e3.dat", usecols=['r_all_rs', 'm_all_rs', 'CNAME', 'CAMPUS']).set_index('CAMPUS').add_suffix('_fourth_last_year')

# Combine the above dataframes, remove duplicate columns
df = pd.concat([df_third_grade_2004, df_fourth_grade_2004, df_fifth_grade_2004, df_fourth_grade_2003], axis=1)
df['CNAME'] = df['CNAME_third'].fillna(df['CNAME_fourth']).fillna(df['CNAME_fifth']).fillna(df['CNAME_fourth_last_year'])
df = df.drop(columns=['CNAME_third', 'CNAME_fourth', 'CNAME_fifth', 'CNAME_fourth_last_year'])
df = df.reset_index()

# display
df


,CAMPUS,r_all_rs_third,m_all_rs_third,r_all_rs_fourth,m_all_rs_fourth,r_all_rs_fifth,m_all_rs_fifth,r_all_rs_fourth_last_year,m_all_rs_fourth_last_year,CNAME
0,1902103,2410.0,2388.0,2392.0,2227.0,2308.0,2317.0,2410.0,2388.0,CAYUGA EL
1,1903101,2256.0,2208.0,2263.0,2192.0,2193.0,2153.0,2256.0,2208.0,ELKHART EL
2,1904102,2284.0,2313.0,2242.0,2152.0,2288.0,2256.0,2284.0,2313.0,FRANKSTON EL
3,1906102,2294.0,2273.0,2218.0,2096.0,2298.0,2312.0,2294.0,2273.0,NECHES EL
4,1907110,2256.0,2205.0,2200.0,2168.0,2218.0,2269.0,2256.0,2205.0,STORY EL
...,...,...,...,...,...,...,...,...,...,...
4232,241904041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WHARTON J H
4233,243902041,NaN,NaN,NaN,NaN,2117.0,2129.0,NaN,NaN,ELECTRA J H
4234,249905102,NaN,NaN,NaN,NaN,2238.0,2255.0,NaN,NaN,DECATUR INT
4235,250907041,NaN,NaN,NaN,NaN,2231.0,2241.0,NaN,NaN,MEMORIAL MIDDLE


In [13]:
# Here is the dataframe above in long format if you'd prefer it that way!
df_long = df.melt(id_vars=['CAMPUS', 'CNAME'], var_name='test_type', value_name='score').dropna()
df_long

,CAMPUS,CNAME,test_type,score
0,1902103,CAYUGA EL,r_all_rs_third,2410.0
1,1903101,ELKHART EL,r_all_rs_third,2256.0
2,1904102,FRANKSTON EL,r_all_rs_third,2284.0
3,1906102,NECHES EL,r_all_rs_third,2294.0
4,1907110,STORY EL,r_all_rs_third,2256.0
...,...,...,...,...
33464,253901101,BENAVIDES EL,m_all_rs_fourth_last_year,2189.0
33465,253901106,ZAPATA SOUTH EL,m_all_rs_fourth_last_year,2253.0
33466,253901107,ZAPATA CENTRAL,m_all_rs_fourth_last_year,2178.0
33467,254901105,LORENZO DE ZAVA,m_all_rs_fourth_last_year,2180.0


# Questions and Answers

Present your findings in this seciton. Keep exploratory code in the scratchpad notebook. 

> _💡 Hint: want to keep this file clean? You can hide away code in functions in `.py` files or `.R` files and just the functions from those files here._

### Question 1: Do we see suspicious activity at Harrell Budd Elementary?
- 👉 Hypothesis (journalistic): We expect to see suspicious activity at Harrell Budd Elementary.
- 👉 Methodology:

In [17]:
# TKTK Code In the Cells Below

#### Answer To Question 1
👉 Write your answer to Question 1 in the cells below. Feel free to include charts as needed. Use complete sentences. Think of this as an internal memo to your editor. They don't have time to sift through a bunch of exploratory analysis, so leave irrelevant info out of your answer

In [14]:
# Your answer goes here

### Question 2: Do we see suspicious activity at other schools?
- 👉 Hypothesis (journalistic): This isn't an isolated phenomenon. Other schools may be doing the same. We aim to identify schools that seem suspicious like Herrald Budd Elementary.
- 👉 Methodology:


#### Answer To Question 2
👉 Write your answer to Question 2 in the cells below. Feel free to include charts as needed. Use complete sentences. Think of this as an internal memo to your editor. They don't have time to sift through a bunch of exploratory analysis, so leave irrelevant info out of your answer

In [16]:
# Your Answer Goes here

# Bonus 🤖 (very very optional)

This loads a giant dataframe with all of the columns from 2004 (there are 2827 columns 😲). 

Here is the documentation for those columns:
https://tea.texas.gov/sites/default/files/TAKS-Variables-prior2005.pdf

How would you use the additional data? What variables will you target first? Design more statstical inquiry that takes advantage of the full dataset.

In [ ]:
df_third_grade_2004 = pd.read_csv("data/cfy04e3.dat").set_index('CAMPUS').assign(grade='third')
df_fourth_grade_2004 = pd.read_csv("data/cfy04e4.dat").set_index('CAMPUS').assign(grade='fourth')
df_fifth_grade_2004 = pd.read_csv("data/cfy04e5.dat").set_index('CAMPUS').assign(grade='fifth')

df_2004 = pd.concat([df_third_grade_2004, df_fourth_grade_2004, df_fifth_grade_2004])
df_2004